In [ ]:
import openai
from openai import OpenAI
from openai.types.beta.threads.message_create_params import Attachment, AttachmentToolFileSearch
import json
import os

In [ ]:
client = OpenAI(api_key="XXX")  

In [ ]:
assistant = client.beta.assistants.create(
    model='gpt-4o-mini',
    instructions="""As a medical assistant specializing in the diagnosis and treatment of cerebral hemorrhage, you are knowledgeable about the following principles:
                        Deep intracerebral hematomas are typically spontaneous.
                        Epidural hematomas are usually caused by trauma.
                        Subdural hematomas are predominantly the result of trauma.
                        Isolated subarachnoid hemorrhage, without other types of cerebral hemorrhage, is likely due to an aneurysm.
                        
                    When formulating treatment plans, you refer to the following guidelines:
                    Spontaneous intracerebral hemorrhage: Refer to the "2022 Guideline for the Management of Patients With Spontaneous Intracerebral Hemorrhage: A Guideline From the American Heart Association/American Stroke Association."
                    Traumatic brain injury: Refer to "Management of Severe Traumatic Brain Injury (first 24 hours)."
                    Aneurysmal subarachnoid hemorrhage: Refer to the "2023 Guideline for the Management of Patients With Aneurysmal Subarachnoid Hemorrhage: A Guideline From the American Heart Association/American Stroke Association."
                        
                    Based on the CT analysis results provided by the user, first determine the cause of the intracerebral hemorrhage. Then, according to the CT analysis results, the cause of the hemorrhage, and the corresponding treatment guidelines for intracerebral hemorrhage, provide additional examinations and treatment recommendations.

                    Request for Recommendations:
                    Additional Testing Recommendations: Based on the CT scan analysis and the guidelines identify any further tests required for a comprehensive assessment of the patient's condition, especially focusing on the possibility of an aneurysm in cases of isolated subarachnoid hemorrhage.
                    Treatment Recommendations: Offer preliminary treatment suggestions, which may include pharmacological therapy, surgical intervention, or other potential measures, tailored to the patient's specific circumstances and in accordance with guideline recommendations.

                    Additional Requirements:
                    All recommendations should strictly adhere to current medical guidelines and best practices.
                    Consider the patient's individual differences to ensure that the recommendations are tailored and adaptable.
                    The provided recommendations should contribute to optimizing the diagnostic and treatment process, aiming to enhance therapeutic outcomes.
                """,
    tools=[{"type": "file_search"}], 
    name='ICH Assistant' 
)

# Create a vector store caled "guidlelines"
vector_store = client.beta.vector_stores.create(name="guidlelines")

# Ready the files for upload to OpenAI
file_paths = ["Management of severe traumatic brain injury.pdf",
              "2022 Guideline for the Management of Patients With Spontaneous Intracerebral Hemorrhage.pdf", 
              '2023 Guideline for the Management of Patients With Aneurysmal Subarachnoid Hemorrhage.pdf']
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
try:
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, 
        files=file_streams
        )
    print(file_batch.status)
    print(file_batch.file_counts)
except Exception as e:
    print(f"Error during file upload: {e}")


assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)


In [ ]:
testnumber = "test"
outputPath = "all_result" + str(testnumber)

# guideline_pdf_file = "2022guideline.pdf"
input_file = 'patients_data.json'
error_log_file = "error_log_" + str(testnumber) + ".txt"


if not os.path.exists(outputPath):
    os.makedirs(outputPath)

In [ ]:
def write_error_log(context, error_msg):
    error_info = f"Context: {context}, Error message: {error_msg}"
    with open(error_log_file, "a", encoding="utf-8") as log_file:
        log_file.write(error_info + "\n")

In [ ]:
with open(input_file, 'r') as file:
    data = json.load(file)

for patient in data:
    patient_id = patient["ID"]
    print(f"\nProcessing patient ID: {patient_id}")
    Intraparenchymal_Hemorrhage_volume = patient["Intraparenchymal_Hemorrhage"]["Volume"]
    Intraparenchymal_Hemorrhage_location = patient["Intraparenchymal_Hemorrhage"]["Location"]
    Intraventricular_Hemorrhage_volume = patient["Intraventricular_Hemorrhage"]["Volume"]
    Intraventricular_Hemorrhage_location = patient["Intraventricular_Hemorrhage"]["Location"]
    Perihematomal_Edema_volume = patient["Perihematomal_Edema"]["Volume"]
    Perihematomal_Edema_location = patient["Perihematomal_Edema"]["Location"]
    Subarachnoid_Hemorrhage_volume = patient["Subarachnoid_Hemorrhage"]["Volume"]
    Subarachnoid_Hemorrhage_location = patient["Subarachnoid_Hemorrhage"]["Location"]
    Subdural_Hemorrhage_volume = patient["Subdural_Hemorrhage"]["Volume"]
    Subdural_Hemorrhage_location = patient["Subdural_Hemorrhage"]["Location"]
    Epidural_Hemorrhage_volume = patient["Epidural_Hemorrhage"]["Volume"]
    Epidural_Hemorrhage_location = patient["Epidural_Hemorrhage"]["Location"]

    prompt = f"""Below is the CT image analysis. 
    
        1. **Intraparenchymal Hemorrhage**:
         - Volume: {Intraparenchymal_Hemorrhage_volume} ml
         - Location: {Intraparenchymal_Hemorrhage_location}
      
        2. **Intraventricular Hemorrhage**:
         - Volume: {Intraventricular_Hemorrhage_volume} ml
         - Location: {Intraventricular_Hemorrhage_location}
      
        3. **Perihematomal Edema**:
         - Volume: {Perihematomal_Edema_volume} ml
         - Location: {Perihematomal_Edema_location}
      
        4. **Subarachnoid Hemorrhage**:
         - Volume: {Subarachnoid_Hemorrhage_volume} ml
         - Location: {Subarachnoid_Hemorrhage_location}
      
        5. **Subdural Hemorrhage**:
         - Volume: {Subdural_Hemorrhage_volume} ml
         - Location: {Subdural_Hemorrhage_location}
      
        6. **Epidural Hemorrhage**:
         - Volume: {Epidural_Hemorrhage_volume} ml
         - Location: {Epidural_Hemorrhage_location}

        Based on the CT analysis results, first determine the cause of the intracerebral hemorrhage. Then, according to the CT analysis results, the cause of the hemorrhage, and the corresponding treatment guidelines for intracerebral hemorrhage, provide additional examinations and treatment recommendations.   
        """

    try:

        thread = client.beta.threads.create()


        client.beta.threads.messages.create(
            thread_id=thread.id,
            role='user',  
            content=prompt 
            )

       
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=assistant.id,
            timeout=600  
        )


        if run.status != "completed":
            raise Exception('Run failed:', run.status)


        messages_cursor = client.beta.threads.messages.list(thread_id=thread.id,run_id=run.id)
        messages = [message for message in messages_cursor]


        treatment_plan = messages[0].content[0].text.value

        output_file = os.path.join(
            outputPath, f"patient_{patient_id}_report.txt")
        with open(output_file, "w", encoding="utf-8") as file:
            file.write(treatment_plan)
        print(f"Saved {output_file} folder")

    except Exception as ex:
        write_error_log(patient_id, f"error: {ex}")
        print(f"error: {ex}")